In [76]:
import pandas as pd
import random
import os
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
# import pickle5 as pickle

from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier


In [15]:
price = pd.read_csv('./price_universe.csv', index_col=0)
price.index = pd.to_datetime(price.index)

In [103]:
snp = price.iloc[:, :1]
train = snp[:-30]
data_train = train.to_numpy()
data_train

array([[2323.76],
       [2339.29],
       [2350.3 ],
       [2366.48],
       [2370.14],
       [2374.89],
       [2373.38],
       [2389.77],
       [2406.38],
       [2400.99],
       [2419.33],
       [2412.86],
       [2425.58],
       [2446.51],
       [2450.18],
       [2451.47],
       [2451.69],
       [2480.38],
       [2462.26],
       [2437.23],
       [2438.31],
       [2435.51],
       [2378.77],
       [2250.2 ],
       [2324.84],
       [2302.41],
       [2237.9 ],
       [2259.91],
       [2291.19],
       [2296.8 ],
       [2327.21],
       [2341.88],
       [2328.72],
       [2339.73],
       [2371.87],
       [2402.72],
       [2370.79],
       [2342.31],
       [2321.39],
       [2345.77],
       [2350.51],
       [2349.86],
       [2363.67],
       [2402.29],
       [2402.29],
       [2388.91],
       [2372.73],
       [2374.24],
       [2375.32],
       [2346.66],
       [2347.31],
       [2342.78],
       [2278.14],
       [2238.93],
       [2292.15],
       [22

In [104]:
test = snp[-30:]
test
data_test = test.to_numpy()
data_test

array([[4744.44],
       [4759.88],
       [4752.8 ],
       [4737.86],
       [4746.72],
       [4757.35],
       [4652.82],
       [4708.75],
       [4622.95],
       [4564.49],
       [4632.57],
       [4593.85],
       [4647.  ],
       [4743.18],
       [4757.35],
       [4724.95],
       [4769.5 ],
       [4726.73],
       [4694.58],
       [4767.98],
       [4726.73],
       [4694.03],
       [4641.6 ],
       [4725.34],
       [4771.42],
       [4801.71],
       [4869.42],
       [4865.61],
       [4871.72],
       [4859.24]])

In [105]:
min_max_scaler = MinMaxScaler()
data_train = min_max_scaler.fit_transform(train.to_numpy().reshape(-1,1))
data_test = min_max_scaler.fit_transform(test.to_numpy().reshape(-1,1))

In [96]:
from torch.utils.data import DataLoader, Dataset

class windowDataset(Dataset):
    def __init__(self, y, input_window=80, output_window=20, stride=5):
        #총 데이터의 개수
        L = y.shape[0]
        #stride씩 움직일 때 생기는 총 sample의 개수
        num_samples = (L - input_window - output_window) // stride + 1

        #input과 output
        X = np.zeros([input_window, num_samples])
        Y = np.zeros([output_window, num_samples])

        for i in np.arange(num_samples):
            start_x = stride*i
            end_x = start_x + input_window
            X[:,i] = y[start_x:end_x]

            start_y = stride*i + input_window
            end_y = start_y + output_window
            Y[:,i] = y[start_y:end_y]

        X = X.reshape(X.shape[0], X.shape[1], 1).transpose((1,0,2))
        Y = Y.reshape(Y.shape[0], Y.shape[1], 1).transpose((1,0,2))
        self.x = X
        self.y = Y
        
        self.len = len(X)
    def __getitem__(self, i):
        return self.x[i], self.y[i, :-1], self.y[i,1:]
    def __len__(self):
        return self.len

In [106]:
iw = 22*12
ow = 22*6

train_dataset = windowDataset(data_train, input_window=iw, output_window=ow, stride=5)
train_loader = DataLoader(train_dataset, batch_size=64)

ValueError: could not broadcast input array from shape (264,1) into shape (264,)